In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

## 读取配置文件

In [ ]:
import yaml
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [4]:
from utils import *
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()
print(validation_dataset[2])  

if not isinstance(train_dataset, list):
        logging.info('Train dataset: %s', train_dataset)

answerable_indices, unanswerable_indices = split_dataset(train_dataset)


if config["dataset"]['answerable_only']:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]

{'id': '56ddde6b9a695914005b962a', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'From which countries did the Norse originate?', 'answers': {'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Ice

## Prompt Engineer

In [5]:
from prompt_engineer import PromptGenerator

promptgenerator = PromptGenerator(config,validation_dataset)

In [6]:
prompt = promptgenerator.generate_template_prompt_by_id(1)
print(prompt)
print(len(promptgenerator.get_unused_indices()))


USER: When were the Normans in Normandy?
ASSISTANT:
5927


In [7]:
prompts = promptgenerator.generate_template_prompts_by_count(2)
print(prompt)
print(len(promptgenerator.get_unused_indices()))

USER: When were the Normans in Normandy?
ASSISTANT:
5925


In [8]:
import random
indices = random.sample(promptgenerator.get_unused_indices(),2)
print(indices)
prompts =promptgenerator.generate_template_prompts_from_indices(indices)
len(promptgenerator.get_unused_indices())

[195, 4955]


5923

In [9]:
example_indices = random.sample(promptgenerator.get_unused_indices(),3)
few_shot_prompt = promptgenerator.construct_fewshot_prompt_from_indices(example_indices)
print(few_shot_prompt)

Answer the following question as briefly as possible.
Context: The origin of electric and magnetic fields would not be fully explained until 1864 when James Clerk Maxwell unified a number of earlier theories into a set of 20 scalar equations, which were later reformulated into 4 vector equations by Oliver Heaviside and Josiah Willard Gibbs. These "Maxwell Equations" fully described the sources of the fields as being stationary and moving charges, and the interactions of the fields themselves. This led Maxwell to discover that electric and magnetic fields could be "self-generating" through a wave that traveled at a speed that he calculated to be the speed of light. This insight united the nascent fields of electromagnetic theory with optics and led directly to a complete description of the electromagnetic spectrum.
Question: How many vector equations did Heaviside and Gibbs reformilate Maxwell's 20 scalar equtions into?
Answer: 4

Answer the following question as briefly as possible.
Co

In [12]:
print(len(promptgenerator.get_unused_indices()))

5920


## model

In [13]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [14]:
huggingface_model = HuggingfaceModel(config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

Generated Text > 
 ('During the 11th and 12th century AD.', [-3.776052474975586, -0.087731271982193, -1.5151315927505493, -0.09163712710142136, -0.6989476084709167, 0.0, -1.1278303861618042, 0.0, 0.0, 0.0, 0.0, -2.4496536254882812, -3.4961366653442383, -0.3642231822013855])


In [16]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

What is the capital of France? Washington
Probability: 0.0080
What is the capital of France? Paris
Probability: 0.2130
What is the capital of France? Beijing
Probability: 0.0314
What is the capital of France? 

Probability: 0.0092


In [17]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)

<Generated Text>: The Mubarak regime?
[-1.094968318939209, -4.177267551422119, -2.4182448387145996, 0.0, 0.0, -0.2782438099384308, -3.365988254547119]


In [18]:
from samplers import Sampler

sampler = Sampler(prompt,huggingface_model,sample_method="simple")

In [19]:
sampler.sample(8)
sampler.show_all_result()

Use the 'simple' Method in the temperature 0.1:
Sample 1: The Normans were in Normandy from 911 to 1066.
Sample 2: In the 11th century.
Sample 3: The Normans were in Normandy from 911 to 1066.
Sample 4: They were there in 1066.
Sample 5: In 1066.
Sample 6: The Normans were in Normandy from 911 to 1066.
Sample 7: In the 11th century.
Sample 8: They were there in 1066.
